# Mitigação de erros de medidas

Consideremos experimentos em um qubit. Consideremos a preparação dos estados da base computacional, $|0\rangle$ ou $|1\rangle$, e a imediata medida subsequente, na base computacional. Como uma primeira aproximação, consideremos que os estados da base computacional são modificados pelos erros de preparação e portas e pelo ruído, sem modificar a pureza/entropia do estado, como segue
\begin{align}
& |0\rangle\ \rightarrow\ R_{00}|0\rangle + R_{10}|1\rangle =: |r_0\rangle, \\
& |1\rangle\ \rightarrow\ R_{01}|0\rangle + R_{11}|1\rangle =: |r_1\rangle.
\end{align}
Podemos dizer então que a base computacional é trasformada pela __matriz de ruído__
$$R = \begin{bmatrix}R_{00}&R_{01} \\ R_{10}&R_{11}\end{bmatrix}$$
como segue
\begin{align}
& R|0\rangle = \begin{bmatrix}R_{00}&R_{01} \\ R_{10}&R_{11}\end{bmatrix}\begin{bmatrix}1 \\ 0\end{bmatrix} = \begin{bmatrix}R_{00} \\ R_{10}\end{bmatrix} = |r_0\rangle, \\ 
& R|1\rangle = \begin{bmatrix}R_{00}&R_{01} \\ R_{10}&R_{11}\end{bmatrix}\begin{bmatrix}0 \\ 1\end{bmatrix} = \begin{bmatrix}R_{01} \\ R_{11}\end{bmatrix} = |r_1\rangle. 
\end{align}
Nesse caso, um estado puro qualquer será transformado por
$$R|\psi\rangle = R(c_0|0\rangle+c_1|1\rangle) = c_0|r_0\rangle+c_1|r_1\rangle =: |r\rangle.$$

Se $\det(R)\ne 0$, a matriz de ruído possui inversa e podemos reverter o seu efeito:
\begin{align}
& |0\rangle = R^{-1}|r_0\rangle, \\
& |1\rangle = R^{-1}|r_1\rangle.
\end{align}
Ou seja
$$|\psi\rangle = R^{-1}|r\rangle.$$

De forma análoga, para um operador densidade o efeito do ruído será
\begin{align}
& \rho = \rho_{00}|0\rangle\langle 0| + \rho_{01}|0\rangle\langle 1| + \rho_{10}|1\rangle\langle 0| + \rho_{11}|1\rangle\langle 1| \\
& \downarrow \\
&  \rho_{00}|r_0\rangle\langle r_0| + \rho_{01}|r_0\rangle\langle r_1| + \rho_{10}|r_1\rangle\langle r_0| + \rho_{11}|r_1\rangle\langle r_1| =: \rho_r = R\rho R^\dagger,
\end{align}
em que assumimos que $R$ pode ser uma matriz complexa. Nesse caso reverteremos o efeito do ruído aplicando
$$\rho = R^{-1}\rho_r (R^\dagger)^{-1}.$$

Com relação a experimentos, o que temos que fazer então é preparar todos os estados da base computacional, $|j\rangle$, e medir na base computacional para obter $|r_j\rangle$. A matriz de ruído é então dada por
$$R = \big[|r_0\rangle\ |r_1\rangle\big].$$
Como tomografia de estados é muito custosa, como uma primeira melhoria reconstruiremos $R$ somente a partir das contagens das medidas. Ou seja, os elementos da matriz de ruído, $R_{jk}$, serão simplesmente as amplitudades de probabilidade de transição $|k\rangle\rightarrow|j\rangle$. Ou seja, para o estado preparado $|j\rangle$, se a contagem do estado $|k\rangle$ é $N_{kj}$ e o número total de medidas (shots) é $N$, teremos que
$$R_{kj} = \sqrt{Pr(|k\rangle\rightarrow|j\rangle)} = \sqrt{\frac{N_{kj}}{N}}.$$

In [83]:
import qiskit
from qiskit import *
nshots = 8192
IBMQ.load_account()
provider= qiskit.IBMQ.get_provider(hub='ibm-q-research-2',group='federal-uni-sant-1',project='main')
#provider = qiskit.IBMQ.get_provider(hub='ibm-q', group='open', project='main')
device = provider.get_backend('ibmq_quito')
simulator = Aer.get_backend('qasm_simulator')
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Options

ibmqfactory.load_account:WARNING:2022-12-19 17:04:47,326: Credentials are already in use. The existing account in the session will be replaced.


In [76]:
#from qiskit import IBMQ, Aer
#provider = IBMQ.load_account()
available_cloud_backends = provider.backends() 
print('\n Cloud backends:')
for i in available_cloud_backends: print(i)

available_local_backends = Aer.backends() 
print('\n Local backends: ')
for i in available_local_backends: print(i)


 Cloud backends:
ibmq_qasm_simulator
ibmq_armonk
ibmq_santiago
ibmq_bogota
ibmq_lima
ibmq_belem
ibmq_quito
simulator_statevector
simulator_mps
simulator_extended_stabilizer
simulator_stabilizer
ibmq_jakarta
ibmq_manila
ibm_lagos
ibm_nairobi
ibm_perth
ibm_oslo

 Local backends: 
aer_simulator
aer_simulator_statevector
aer_simulator_density_matrix
aer_simulator_stabilizer
aer_simulator_matrix_product_state
aer_simulator_extended_stabilizer
aer_simulator_unitary
aer_simulator_superop
qasm_simulator
statevector_simulator
unitary_simulator
pulse_simulator


In [84]:
QiskitRuntimeService.save_account(channel='ibm_quantum', 
                                  token='17efde49764005e8eeb00dd065d44bc208778be72d44b475e508d20504818786f842988b0e506515c78debdd1b0c4b570717863db5e4f85569fb43c4c8626b8a',
                                  overwrite=True)
service = QiskitRuntimeService(channel='ibm_quantum',instance='ibm-q/open/main')
#service = QiskitRuntimeService(channel='ibm_quantum',instance='ibm-q-research-2/federal-uni-sant-1/main')

In [85]:
nqr = 1; ncr = 1
qr = QuantumRegister(nqr); cr = ClassicalRegister(ncr)

qc0 = QuantumCircuit(qr,cr); qc1 = QuantumCircuit(qr,cr)                     
qc0.measure(qr[0],cr[0])
#display(qc0.draw("mpl"))

qc1.x(qr[0])
qc1.measure(qr[0],cr[0])
#display(qc1.draw("mpl"))

options = Options()
#options.optimization_level = 2
#options.resilience_level = 0
service = QiskitRuntimeService()
with Session(service=service, backend=device):
    sampler = Sampler()
    job = sampler.run([qc0,qc1])
    #print(job.job_id())
result = job.result()
print(result.quasi_dists, result.quasi_dists[0], result.quasi_dists[0][0])

QiskitBackendNotFoundError: 'Backend ibmq_quito cannot be found in any hub/group/project for this account.'

In [ ]:
R = np.zeros((2,2))#; R


In [ ]:
qr = QuantumRegister(1); cr = ClassicalRegister(1); qc = QuantumCircuit(qr,cr)
qc.measure(qr[0],cr[0])
qc.draw()
job = qiskit.execute(qc, backend = device, shots=nshots)
print(job.job_id())
job_monitor(job)

In [ ]:
#job.
print(job.result().data())
#plot_histogram(job.result())